In [1]:
import os
import re
import pandas as pd

## BTG

In [2]:
def create_output_directory(output_directory):
    if not os.path.exists(output_directory):
        os.makedirs(output_directory)

def read_excel_file(file_path):
    df = pd.read_excel(file_path)
    temp_csv_path = os.path.join(os.path.dirname(file_path), "temp.csv")
    df.to_csv(temp_csv_path, index=False, header=True)
    df = pd.read_csv(temp_csv_path)
    os.remove(temp_csv_path)
    return df

def process_portfolio_investido(df, nome_fundo, data):
    try:
        start_index = df[df[df.columns[0]] == 'Portfolio_Investido'].index[0]
        end_index = df[df[df.columns[0]] == 'DESPESAS'].index[0]
        portfolio_df = df.loc[start_index:end_index].iloc[1:-3].reset_index(drop=True)

        new_column_names = portfolio_df.iloc[0].tolist()
        portfolio_df.columns = new_column_names
        portfolio_df = portfolio_df[1:].reset_index(drop=True)
        portfolio_df.insert(0, 'Nome Fundo', nome_fundo)
        portfolio_df.insert(1, 'Data', data)
        portfolio_df = portfolio_df.iloc[:, :9].drop(columns=['ISIN', 'CNPJ', '% P.L.'])
        portfolio_df['Classificacao'] = 'PORTFOLIO INVESTIDO'
        
        return portfolio_df, new_column_names
    except Exception as e:
        print(f"Erro ao processar Portfolio Investido do fundo '{nome_fundo}': {e}")
        return None, None

def process_titulos_publicos(df, nome_fundo, data, new_column_names):
    try:
        start_index = df[df[df.columns[0]] == 'Titulos_Publicos'].index[0]
        end_index = df[df[df.columns[0]].isna() & (df.index > start_index)].index[0]
        titulos_df = df.loc[start_index:end_index-1].iloc[1:].reset_index(drop=True)
        titulos_df.columns = new_column_names
        titulos_df['CNPJ'] = ''
        titulos_df['Quantidade'] = ''
        titulos_df['Quota'] = ''
        titulos_df['Portfólio Inv.'] = titulos_df['Financeiro']
        titulos_df['Financeiro'] = titulos_df['Var.Diária']
        titulos_df.iloc[:, titulos_df.columns.get_loc('% P.L.')] = titulos_df.iloc[:, -2]
        titulos_df = titulos_df.drop(columns=['ISIN']).iloc[:, :6].drop(index=0)
        titulos_df.insert(0, 'Nome Fundo', nome_fundo)
        titulos_df.insert(1, 'Data', data)
        titulos_df = titulos_df.drop(columns=['CNPJ', '% P.L.'])
        titulos_df['Classificacao'] = 'TITULOS PUBLICOS'
        
        return titulos_df
    except Exception as e:
        print(f"Erro ao processar Títulos Públicos do fundo '{nome_fundo}': {e}")
        return None

def process_acoes(df, nome_fundo, data, new_column_names):
    try:
        start_index = df[df[df.columns[0]] == 'Acoes'].index[0]
        end_index = df[df[df.columns[0]].isna() & (df.index > start_index)].index[0]
        acoes_df = df.loc[start_index:end_index-1].iloc[1:].reset_index(drop=True)
        acoes_df.columns = new_column_names
        acoes_df['Portfólio Inv.'] = acoes_df['Quantidade']
        acoes_df['Quantidade'] = acoes_df['Quota']
        acoes_df['Quota'] = acoes_df['Financeiro']
        acoes_df['Financeiro'] = acoes_df['% P.L.']
        acoes_df = acoes_df.drop(columns=['ISIN', 'CNPJ', '% P.L.']).iloc[:, :4].drop(index=0)
        acoes_df.insert(0, 'Nome Fundo', nome_fundo)
        acoes_df.insert(1, 'Data', data)
        acoes_df['Classificacao'] = 'ACOES'
        
        return acoes_df
    except Exception as e:
        print(f"Erro ao processar Ações do fundo '{nome_fundo}': {e}")
        return None

def process_despesas(df, nome_fundo, data):
    try:
        start_index = df[df[df.columns[0]] == 'DESPESAS'].index[0]
        despesas_df = df.loc[start_index:].iloc[1:, :4]
        new_column_names = despesas_df.iloc[0].tolist()
        despesas_df.columns = new_column_names
        despesas_df = despesas_df[1:].reset_index(drop=True)
        despesas_df = despesas_df.rename(columns={'Nome': 'Portfólio Inv.', 'Valor': 'Financeiro'})
        despesas_df = despesas_df.drop(columns=['Data Início Vigência','Data Fim Vigência'])
        despesas_df.insert(0, 'Nome Fundo', nome_fundo)
        despesas_df.insert(1, 'Data', data)
        despesas_df['Classificacao'] = 'DESPESAS'
        
        return despesas_df
    except Exception as e:
        print(f"Erro ao processar Despesas do fundo '{nome_fundo}': {e}")
        return None

def extract_and_format_data(df):
    try:
        nome_fundo = df.iloc[5, 0].replace('_', ' ')
        data = df.iloc[6, 1]
        
        portfolio_df, new_column_names = process_portfolio_investido(df, nome_fundo, data)
        
        if portfolio_df is None:
            return data, None
        
        titulos_df = process_titulos_publicos(df, nome_fundo, data, new_column_names) if 'Titulos_Publicos' in df[df.columns[0]].values else None
        acoes_df = process_acoes(df, nome_fundo, data, new_column_names) if 'Acoes' in df[df.columns[0]].values else None
        despesas_df = process_despesas(df, nome_fundo, data) if 'DESPESAS' in df[df.columns[0]].values else None

        # Formata as colunas diretamente no dataframe final
        colunas_para_formatar = ['Quantidade', 'Quota', 'Financeiro']
        final_df = pd.concat([portfolio_df, titulos_df, acoes_df, despesas_df], ignore_index=True)
        for coluna in colunas_para_formatar:
            if coluna in final_df.columns:
                final_df[coluna] = final_df[coluna].astype(str).str.replace('.', ',')
        final_df = final_df.astype(str).replace('nan', '').fillna('')
        
        return data, final_df
    
    except Exception as e:
        print(f"Erro ao extrair e formatar dados do fundo '{nome_fundo}': {e}")
        return None, None

def save_dataframes_by_date(dataframes_by_date, output_directory):
    for date, dfs in dataframes_by_date.items():
        concatenated_df = pd.concat(dfs, ignore_index=True)
        output_file_name = f'CarteiraDiaria_Concatenated_{date.replace("/", "_")}.csv'
        output_file_path = os.path.join(output_directory, output_file_name)
        concatenated_df.to_csv(output_file_path, index=False)
        print(f"Arquivos concatenados salvos em '{output_file_path}'")

def process_files(input_directory, output_directory):
    create_output_directory(output_directory)
    dataframes_by_date = {}
    files = [f for f in os.listdir(input_directory) if f.endswith('.xlsx')]
    
    for file in files:
        file_path = os.path.join(input_directory, file)
        df = read_excel_file(file_path)
        data, final_df = extract_and_format_data(df)
        
        if data not in dataframes_by_date:
            dataframes_by_date[data] = []
        dataframes_by_date[data].append(final_df)
    
    save_dataframes_by_date(dataframes_by_date, output_directory)


In [3]:
# Definição dos diretórios de entrada e saída
input_directory = r"\\172.31.8.209\Files\CataliseInvestimentos\11. Gestao (DCM, Riscos,Dados)\11.12 Analises de Risco\Fundos_monitorados_FIC\Carteira Diária Locação\CarteiraDiariaBTG"
output_directory = r"\\172.31.8.209\Files\CataliseInvestimentos\11. Gestao (DCM, Riscos,Dados)\11.12 Analises de Risco\Fundos_monitorados_FIC\Carteira Diária Locação\OutputBTG"

# Execução do processamento dos arquivos
process_files(input_directory, output_directory)

Arquivos concatenados salvos em '\\172.31.8.209\Files\CataliseInvestimentos\11. Gestao (DCM, Riscos,Dados)\11.12 Analises de Risco\Fundos_monitorados_FIC\Carteira Diária Locação\OutputBTG\CarteiraDiaria_Concatenated_11_03_2025.csv'
Arquivos concatenados salvos em '\\172.31.8.209\Files\CataliseInvestimentos\11. Gestao (DCM, Riscos,Dados)\11.12 Analises de Risco\Fundos_monitorados_FIC\Carteira Diária Locação\OutputBTG\CarteiraDiaria_Concatenated_12_03_2025.csv'
Arquivos concatenados salvos em '\\172.31.8.209\Files\CataliseInvestimentos\11. Gestao (DCM, Riscos,Dados)\11.12 Analises de Risco\Fundos_monitorados_FIC\Carteira Diária Locação\OutputBTG\CarteiraDiaria_Concatenated_13_03_2025.csv'
Arquivos concatenados salvos em '\\172.31.8.209\Files\CataliseInvestimentos\11. Gestao (DCM, Riscos,Dados)\11.12 Analises de Risco\Fundos_monitorados_FIC\Carteira Diária Locação\OutputBTG\CarteiraDiaria_Concatenated_14_03_2025.csv'
Arquivos concatenados salvos em '\\172.31.8.209\Files\CataliseInvestimen

## Daycoval

In [4]:
# Define o caminho da pasta de entrada e de saída
input_directory = r"\\172.31.8.209\Files\CataliseInvestimentos\11. Gestao (DCM, Riscos,Dados)\11.12 Analises de Risco\Fundos_monitorados_FIC\Carteira Diária Locação\CarteiraDiariaDaycoval"
output_directory = r"\\172.31.8.209\Files\CataliseInvestimentos\11. Gestao (DCM, Riscos,Dados)\11.12 Analises de Risco\Fundos_monitorados_FIC\Carteira Diária Locação\OutputDaycoval"

# Lista dos nomes antigos e novos para a coluna 'NomeFundo'
nomes_antigos = ['AVANT FINTECH FIC FIM', 'GRUPO PRIME AGRO FIC FIM RESPONSABILIDAD', 'GOLIATH FUNDO DE INVESTIMENTO MULTIMERCA',
                 'ARTANIS FUNDO DE INVESTIMENTO MULTIMERCA', 'ARCTURUS FIM CP IE', 'ABIATAR FIC DE FIM RESPONSABILIDADE LIMI', 
                 'NINE CAPITAL FIC DE FIM CP', 'MF HOLDING GROUP FIC FIM', 'BELLIN FIC FIM RESPONSABILIDADE LIMTADA', 'LC AZUL FIC DE FIM RESP LIMITADA']

nomes_novos = ['FICFIM AVANT','FIC PRIME AGRO','FIC GOLIATH','FIC ARTANIS','FIC ARCTURUS','FIC ABIATAR','FICFIM NINE','FICFIM MF HOLDI', 
               'FICFIM BELLIN', 'FICFIM LC AZUL']  

# Cria um dicionário de mapeamento dos nomes antigos para os novos nomes
mapeamento_nomes = dict(zip(nomes_antigos, nomes_novos))

codigos = [
    '4471709', '8351082', '8367710', '8367779', '8618178', '8696977', '8745218', '8935297', '8935386', '8935564', '8935572', 
    '8881480', '8936315', '8936323', '9098470', '8935572', '9101063', '9674462'
]

# Cria o diretório de saída se não existir
if not os.path.exists(output_directory):
    os.makedirs(output_directory)

# Itera sobre todos os arquivos CSV na pasta de entrada
for filename in os.listdir(input_directory):
    if filename.endswith(".csv"):
        # Caminho completo do arquivo CSV
        csv_path = os.path.join(input_directory, filename)

        # Lê o arquivo CSV em um DataFrame com encoding 'latin1' e separador ';'
        df = pd.read_csv(csv_path, encoding='latin1', sep=";", header=None)

        # Remove a última linha e a primeira coluna
        df = df.iloc[:-1, 1:]

        # Define os novos nomes das colunas com base na última linha do DataFrame
        new_column_names = df.iloc[-1].tolist()
        df.columns = new_column_names

        # Filtra para manter apenas as linhas onde a primeira coluna é 'LayCrtDia' - desconsiderar linha do CDI
        df = df[df.iloc[:, 0] == 'LayCrtDia']
        df = df[df['CodigoCrt'].isin(codigos)]
        
        # Filtra para manter apenas as linhas onde a segunda coluna é 'V'
        df = df[df.iloc[:, 1] == 'V']

        # # Dropar as colunas que têm apenas valores NaN
        # df = df.dropna(axis=1, how='all')
        # df

        # Define a lista de colunas a serem mantidas e seus novos nomes
        colunas = ['NomeCrt', 'TpCrt', 'DataEmis', 'Nome', 'Espec', 'Qt', 'VlMrc', 'DataVenc', 'DataAplic', 'NoGrpN2']
        novos_nomes_colunas = ['NomeFundo', 'TipoFundo', 'DtPosicao', 'Nome', 'Espec', 'Qnt', 'VlrMercado', 'DataVenc', 'DataAplic', 'Classificacao']

        # Filtra o DataFrame para manter apenas as colunas especificadas
        df = df[colunas]
        
        # Renomeia as colunas do DataFrame
        df.columns = novos_nomes_colunas

        # Converte os valores das colunas 'Nome' e 'Espec' para maiúsculas
        df['Nome'] = df['Nome'].str.upper()
        df['Espec'] = df['Espec'].str.upper()

        # Remove caracteres em branco no final de todos os valores da tabela
        df = df.map(lambda x: x.strip() if isinstance(x, str) else x)

        # Renomeia os valores na coluna 'NomeFundo' com base no mapeamento
        df['NomeFundo'] = df['NomeFundo'].replace(mapeamento_nomes)
        
        # Obtém a primeira data de emissão da coluna 'DtPosicao'
        data_emissao = df['DtPosicao'].iloc[0]
        # Converte a data para o formato YYYYMMDD para usar no nome do arquivo
        data_emissao_str = pd.to_datetime(data_emissao, dayfirst=True).strftime('%Y%m%d')
        
        # Renomeia o arquivo de entrada
        new_input_filename = f"CD_Daycoval_{data_emissao_str}.csv"
        new_csv_path = os.path.join(input_directory, new_input_filename)
        os.rename(csv_path, new_csv_path)
        
        # Define o caminho do arquivo de saída
        output_file_path = os.path.join(output_directory, f"CarteiraDiaria_{data_emissao_str}.csv")

        # Salva o DataFrame no arquivo CSV com o nome especificado
        df.to_csv(output_file_path, index=False)

        print(f"DataFrame salvo em '{output_file_path}' e arquivo de entrada renomeado para '{new_input_filename}'")
df.info()

IndexError: single positional indexer is out-of-bounds